In [1]:
#ADD TO THIS FOLDER WHEN YOU ARE SURE

In [2]:
#USE THIS CODE IF YOU GET LOST OR ADD STUFF THAT YOU FEEL WOULD BE USEFUL FOR EVERYONE

In [3]:
#IF IMPORTS FAIL, IMPORT THE FOLLOWING
#! pip install pyspark
#! pip install snorkel
#! pip install wordcloud

In [ ]:
# Import Libraries, OS and Setup 
%matplotlib inline
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
#from wordcloud import WordCloud
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from pyspark.sql import SQLContext

In [ ]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
import pyspark

number_cores = 8
memory_gb = 24
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
sc = pyspark.SparkContext(conf=conf)

# 1. Loading Data
Read Dataset

Split Data

In [ ]:
# Import the Data
sqlContext = SQLContext(sc)

amazon_reviews_dataset_sql_context = sqlContext.read.csv("/project/DE_Group_Project_2020/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv",header=True)
amazon_reviews_dataset_pd_df = pd.read_csv("/project/DE_Group_Project_2020/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")
amazon_reviews_dataset_rdd = amazon_reviews_dataset_sql_context.rdd

# 2. Writing Labeling Functions

a) Exploring the training set for initial ideas

b) Writing LFs

c) Evaluate performance on training set

d) Balance accuracy and coverage

e) Writing an LF that uses a third-party model

Next, for each rating tier, we want to visualize the words that seem to appear the most. These most common words will then be used to determine whether positive reviews are contained in the last 3 rating tiers and negative reviews are contained in the first two rating tiers.
The first rating tier (1) is considered to be consistent of really negative reviews. The following graph, helps us understand the words that seem to appear in this specific tier.

In [ ]:
# Visualise the 5 different reviews WordCloud and figure other ways to quantify frequency of words
# From - https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73

# WorldCloud - Review == 1 
reviewone = ' '.join(list(amazon_reviews_dataset_pd_df[amazon_reviews_dataset_pd_df['reviews.rating']==1]['reviews.text']))
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

The second rating tier (2) is considered to be consistent of less negative reviews compared to the first tier. The following graph, helps us understand the words that seem to appear in this specific tier.

In [ ]:
# WorldCloud - Review == 2 
reviewone = ' '.join(list(amazon_reviews_dataset_pd_df[amazon_reviews_dataset_pd_df['reviews.rating']==2]['reviews.text']))
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

The third rating tier (3) is considered to be consistent of positive reviews but not that pleasing . The following graph, helps us understand the words that seem to appear in this specific tier.

In [ ]:
# WorldCloud - Review == 3 
reviewone = ' '.join(list(amazon_reviews_dataset_pd_df[amazon_reviews_dataset_pd_df['reviews.rating']==3]['reviews.text']))
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

The fourth rating tier (4) is considered to be consistent of more positive reviews compared to the third tier. The following graph, helps us understand the words that seem to appear in this specific tier.

In [2]:
# WorldCloud - Review == 4 
reviewone = ' '.join(list(amazon_reviews_dataset_pd_df[amazon_reviews_dataset_pd_df['reviews.rating']==4]['reviews.text']))
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

NameError: name 'amazon_reviews_dataset_pd_df' is not defined

The fifth rating tier (5) is considered to be consistent of the most positive reviews. The following graph, helps us understand the words that seem to appear in this specific tier.

In [ ]:
# WorldCloud - Review == 5
reviewone = ' '.join(list(amazon_reviews_dataset_pd_df[amazon_reviews_dataset_pd_df['reviews.rating']==5]['reviews.text']))
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

We define constants to represent the class labels for:
- Rating one = 0
- Rarting two = 1
- Rarting three = 2
- Rating four = 3
- Rating five = 4
- Abstaining: -1

In [ ]:
# Set voting values.
RATINGONE = 0
RATINGTWO = 1 
RATINGTHREE = 2
RATINGFOUR = 3
RATINGFIVE = 4

ABSTAIN = -1

We will now write labelling functions to determine whether positive reviews are contained in the last three rating tiers and negative reviews are contained in the first two rating tiers as expected by human intuition.

In [1]:
# Common posititve words 
POSITIVE = r"\bjew (love|great|good|easy|happy|nice|fun|pretty|works|better|well|ok)"
# Common negative words
NEGATIVE = r"\bjew (old|junk|useless|failded|slow|nothing)"

# labelling functions for five different rating tiers 
@labeling_function()
def positive(x):
  return RATINGTHREE, RATINGFOUR, RATINGFIVE if POSITIVE in x.text.lower() else ABSTAIN

@labeling_function()
def negative(x):
 return RATINGONE, RATINGTWO if NEGATIVE in x.text.lower() else ABSTAIN

NameError: name 'labeling_function' is not defined

To apply the created labelling functions to a collection of data points, we use an LFApplier. In this case, we use the SparkLFApplier because our data points are represented with a RDD.

In [ ]:
# Label matrix

from snorkel.labeling.apply.spark import SparkLFApplier

lfs = [positive, negative]

applier = SparkLFApplier(lfs=lfs)
L_train = applier.apply(data_points = amazon_reviews_dataset_rdd)

We calculate the coverage of the created labelling functions, the percentage of the dataset that they label in order to evaluate the performance of our labels.

In [ ]:
coverage_positive, coverage_negative = (L_train != ABSTAIN).mean(axis=0)
print(f"positive coverage: {coverage_positive * 100:.1f}%")
print(f"negative coverage: {coverage_negative * 100:.1f}%")

In [ ]:
LFAnalysis(L=applied, lfs=labelingfunctions).lf_summary()

# 3. Writing More Labeling Functions

a) Keyword LFs

b) Pattern-matching LFs (regular expressions)

c) Heuristic LFs

d) LFs with Complex Preprocessors

We will now move forward with out analysis and start writting labelling functions that use a third-party model.
The TextBlob tool suppliers a pretrained sentiment analyzer. The positive and negative classification tasks are not the same as sentiment classifications. We might think that rating tiers have different distributions of sentiment scores (Snorkel.org, 2020).
We start by developing a Preprocessor that runs TextBlob on our review comments and later, obtains the polarity and subjectivity scores.

In [ ]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob

# extract polarity and subjectivity scores 
@preprocessor(memoize=True)
def textblob_sentiment(x):
    scores = TextBlob(x.text)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

First, we focus on writing labelling functions for the last three rating tiers which are though to contain mostly positive reviews comments.

In [ ]:
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return RATINGTHREE, RATINGFOUR, RATINGFIVE if x.polarity > 0.9 else ABSTAIN

In [ ]:
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return RATINGTHREE, RATINGFOUR, RATINGFIVE if x.subjectivity >= 0.5 else ABSTAIN

We now need to apply our labelling functions so we can analyze their performance.

In [ ]:
from snorkel.labeling.apply.spark import SparkLFApplier
lfs = [textblob_polarity, textblob_subjectivity]

applier = SparkLFApplier(lfs)
applied = applier.apply(data_points=amazon_reviews_dataset_rdd)

In [ ]:
LFAnalysis(L=applied, lfs=labelingfunctions).lf_summary()

Second, we focus on writing labelling functions for the first two rating tiers which are though to contain mostly negative reviews comments.


In [ ]:
@labeling_function(pre=[textblob_sentiment])
def textblob_polarity(x):
    return RATINGONE, RATINGTWO if x.polarity <= 0.9 else ABSTAIN

In [ ]:
@labeling_function(pre=[textblob_sentiment])
def textblob_subjectivity(x):
    return RATINGONE, RATINGTWO if x.subjectivity < 0.5 else ABSTAIN

Then, we to apply our labelling functions so we can analyze their performance.

In [ ]:
lfs = [textblob_polarity, textblob_subjectivity]

applier = PandasLFApplier(lfs)
L_train = applier.apply(amazon_reviews_dataset_pd_df)

# 4. Combining Labeling Function Outputs with the Label Model

# 5. Evaluate Performance on Training Set

# 6. Filtering out unlabeled data points

# 7. Training a Classifier

Featurization

Scikit-Learn Classifier

# 7. Summary

Data Pipeline